# To find the segement for financial service in New York

### Build the required environment 
All of the package would be used in this sequence including numpy, pandas, geopy, matplotlib and folium to manipulate the dataset and add the marker on the map. Package "sklearn" will launch the K-means analysis.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')

Libraries imported.


### Load the map data
Using the public sources to pickout the borough / neighbourhood name.
The address of the resources: https://cocl.us/new_york_dataset.

Create the table of neighourhood and its location information.

In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [4]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Use geopy library to get the latitude and longitude values of New York City

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [5]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create a map of New York with neighborhoods superimposed on top

In [6]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [7]:
import folium
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### Foursquare venues
import the data from Foursquare to find out the venues around the financial services in the New York City.

In [8]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [9]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = 'SO1VLL3V3U4EXE1YXKWQWJPC1X01P5HUC3DYWCQJVWR3ELFQ'
CLIENT_SECRET = 'NFAZOOPXIQHM24LSRK2RF4SNS5IQGEAV2G4IGQKQ1VY5C530'
VERSION = '20180605'

In [10]:
#https://developer.foursquare.com/docs/resources/categories
#Finance = 503287a291d4c4b30a586d65
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_finance = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='503287a291d4c4b30a586d65')
newyork_venues_finance.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,ATAX,40.877771,-73.905196,Financial or Legal Service
1,Marble Hill,40.876551,-73.91066,Capital One Bank,40.885985,-73.910183,Bank
2,Marble Hill,40.876551,-73.91066,Bank of America,40.884358,-73.900716,Bank
3,Marble Hill,40.876551,-73.91066,Liberty Tax,40.872947,-73.906740,Financial or Legal Service
4,Marble Hill,40.876551,-73.91066,PAYOMATIC,40.874833,-73.909956,Check Cashing Service


In [11]:
newyork_venues_finance.shape

(772, 7)

In [12]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [13]:
map_newyork_finance = folium.Map(location=[latitude, longitude], zoom_start=13)
addToMap(newyork_venues_finance, 'red', map_newyork_finance)

map_newyork_finance

In [14]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [15]:
manhattan_grouped = newyork_venues_finance.groupby('Neighborhood').count()
manhattan_grouped
print('There are {} uniques categories.'.format(len(newyork_venues_finance['Venue Category'].unique())))

There are 14 uniques categories.


### Analyze Each Neighborhood
To find out the top 10 venues in each neighorbood and sorted in the ascending order.

In [16]:
manhattan_finance = pd.get_dummies(newyork_venues_finance[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_finance['Neighborhood'] = newyork_venues_finance['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_finance.columns[-1]] + list(manhattan_finance.columns[:-1])
manhattan_finance = manhattan_finance[fixed_columns]

manhattan_finance.head()

,Neighborhood,ATM,Advertising Agency,Bank,Building,Check Cashing Service,Community Center,Financial or Legal Service,Food Court,Insurance Office,Lawyer,Non-Profit,Office,Real Estate Office,Travel Agency
0,Marble Hill,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,Marble Hill,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,Marble Hill,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,Marble Hill,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Marble Hill,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [17]:
manhattan_grouped = manhattan_finance.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,ATM,Advertising Agency,Bank,Building,Check Cashing Service,Community Center,Financial or Legal Service,Food Court,Insurance Office,Lawyer,Non-Profit,Office,Real Estate Office,Travel Agency
0,Battery Park City,0.058824,0.0,0.411765,0.000000,0.000000,0.00000,0.470588,0.0,0.0,0.000000,0.000000,0.058824,0.0,0.0
1,Carnegie Hill,0.000000,0.0,0.444444,0.000000,0.000000,0.00000,0.555556,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
2,Central Harlem,0.000000,0.0,0.156250,0.000000,0.031250,0.03125,0.781250,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0
3,Chelsea,0.000000,0.0,0.210526,0.000000,0.052632,0.00000,0.631579,0.0,0.0,0.052632,0.052632,0.000000,0.0,0.0
4,Chinatown,0.000000,0.0,0.375000,0.041667,0.041667,0.00000,0.541667,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0


In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Financial or Legal Service,Bank,Office,ATM,Travel Agency,Real Estate Office,Non-Profit,Lawyer,Insurance Office,Food Court
1,Carnegie Hill,Financial or Legal Service,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court,Community Center
2,Central Harlem,Financial or Legal Service,Bank,Community Center,Check Cashing Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
3,Chelsea,Financial or Legal Service,Bank,Non-Profit,Lawyer,Check Cashing Service,Travel Agency,Real Estate Office,Office,Insurance Office,Food Court
4,Chinatown,Financial or Legal Service,Bank,Check Cashing Service,Building,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office


### Cluster Neighborhoods

In [20]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 2, 4, 1, 3, 0, 2, 3, 1], dtype=int32)

In [21]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Financial or Legal Service,Bank,Check Cashing Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court
1,Manhattan,Chinatown,40.715618,-73.994279,1,Financial or Legal Service,Bank,Check Cashing Service,Building,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Financial or Legal Service,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court,Community Center
3,Manhattan,Inwood,40.867684,-73.921210,4,Financial or Legal Service,Bank,Check Cashing Service,Insurance Office,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Food Court
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Financial or Legal Service,Check Cashing Service,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [23]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Clinton,Financial or Legal Service,Lawyer,Check Cashing Service,Building,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Insurance Office
39,Hudson Yards,Financial or Legal Service,Lawyer,Non-Profit,Check Cashing Service,Travel Agency,Real Estate Office,Office,Insurance Office,Food Court,Community Center


In [24]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Financial or Legal Service,Bank,Check Cashing Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court
1,Chinatown,Financial or Legal Service,Bank,Check Cashing Service,Building,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
2,Washington Heights,Financial or Legal Service,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court,Community Center
12,Upper West Side,Financial or Legal Service,Bank,Insurance Office,ATM,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Food Court
16,Murray Hill,Financial or Legal Service,Bank,Lawyer,Office,Insurance Office,Building,Travel Agency,Real Estate Office,Non-Profit,Food Court
21,Tribeca,Financial or Legal Service,Bank,Check Cashing Service,Building,ATM,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer
25,Manhattan Valley,Financial or Legal Service,Bank,Check Cashing Service,ATM,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
28,Battery Park City,Financial or Legal Service,Bank,Office,ATM,Travel Agency,Real Estate Office,Non-Profit,Lawyer,Insurance Office,Food Court
29,Financial District,Financial or Legal Service,Bank,Office,ATM,Travel Agency,Real Estate Office,Non-Profit,Lawyer,Insurance Office,Food Court
30,Carnegie Hill,Financial or Legal Service,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court,Community Center


In [25]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Hamilton Heights,Financial or Legal Service,Check Cashing Service,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court
5,Manhattanville,Financial or Legal Service,Bank,Check Cashing Service,Community Center,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
6,Central Harlem,Financial or Legal Service,Bank,Community Center,Check Cashing Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
7,East Harlem,Financial or Legal Service,Bank,Check Cashing Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court
37,Stuyvesant Town,Financial or Legal Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court,Community Center,Check Cashing Service


In [26]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Bank,Financial or Legal Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court,Community Center
9,Yorkville,Financial or Legal Service,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court,Community Center
10,Lenox Hill,Bank,Financial or Legal Service,Food Court,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Community Center
18,Greenwich Village,Bank,Financial or Legal Service,Non-Profit,Check Cashing Service,Building,Travel Agency,Real Estate Office,Office,Lawyer,Insurance Office
19,East Village,Financial or Legal Service,Bank,Check Cashing Service,Advertising Agency,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
22,Little Italy,Bank,Financial or Legal Service,Check Cashing Service,Building,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
23,Soho,Financial or Legal Service,Bank,Check Cashing Service,Building,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
27,Gramercy,Bank,Financial or Legal Service,Lawyer,Check Cashing Service,Travel Agency,Real Estate Office,Office,Non-Profit,Insurance Office,Food Court
31,Noho,Bank,Financial or Legal Service,Check Cashing Service,Building,Advertising Agency,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer
32,Civic Center,Bank,Financial or Legal Service,Office,ATM,Travel Agency,Real Estate Office,Non-Profit,Lawyer,Insurance Office,Food Court


In [27]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Inwood,Financial or Legal Service,Bank,Check Cashing Service,Insurance Office,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Food Court
11,Roosevelt Island,Financial or Legal Service,Bank,Food Court,Check Cashing Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office
13,Lincoln Square,Financial or Legal Service,Bank,Insurance Office,Office,Check Cashing Service,ATM,Travel Agency,Real Estate Office,Non-Profit,Lawyer
15,Midtown,Financial or Legal Service,Bank,Office,Lawyer,Check Cashing Service,Travel Agency,Real Estate Office,Non-Profit,Insurance Office,Food Court
17,Chelsea,Financial or Legal Service,Bank,Non-Profit,Lawyer,Check Cashing Service,Travel Agency,Real Estate Office,Office,Insurance Office,Food Court
20,Lower East Side,Financial or Legal Service,Bank,Check Cashing Service,Office,Advertising Agency,Travel Agency,Real Estate Office,Non-Profit,Lawyer,Insurance Office
24,West Village,Financial or Legal Service,Bank,Check Cashing Service,Real Estate Office,Non-Profit,Building,Travel Agency,Office,Lawyer,Insurance Office
26,Morningside Heights,Financial or Legal Service,Bank,Check Cashing Service,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court
33,Midtown South,Financial or Legal Service,Bank,Lawyer,Check Cashing Service,Office,Building,Travel Agency,Real Estate Office,Non-Profit,Insurance Office
34,Sutton Place,Financial or Legal Service,Bank,Travel Agency,Real Estate Office,Office,Non-Profit,Lawyer,Insurance Office,Food Court,Community Center
